# MLE Capstone: Outfit Recommender - Spring 2021
# Step 2: Data Collection & Initial EDA of Candidate Datasets
# Dataset 1: iMaterialist

### By: Bazeley, Mikiko 
### GH: @mmbazel  

In this notebook, I'll be exploring one of three datasets (DeepFashion, Paperdoll, and iMaterialist). 

Specifically in this notebook we'll: 
* Load the data
* Explore the dimensions of the dataset
* Understand what categories are being represented

## Step 1: Proper set-up & installation of necessary libraries & packages

1. Ensure you've set-up with the Kaggle API
* Pro Tip: If you're trying to set up the path to the kaggle.json file as an environment variable on MacOS (Catalina), easiest way is to set at the command line. It was annoying dealing with the different editors like vim & nano. 

$ KAGGLE_CONFIG_DIR= {INSERT_YOUR_HOMEPATH_HERE}/.kaggle export KAGGLE_CONFIG_DIR


In [2]:
##################### [TODO] SETUP #####################
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import zipfile
import time

%matplotlib inline

print('Packages Imported')


# [TODO] Package install/load
# [TODO] Import any utilities functions
# [TODO] Set any settings
# [TODO] Load any secrets/credetials 

Packages Imported


In [ ]:
# Load Kaggle credentials

KAGGLE_CONFIG_DIR = os.getenv('KAGGLE_CONFIG_DIR')
kaggle_credential_filename = 'kaggle.json'

with open(f'{KAGGLE_CONFIG_DIR}\\{kaggle_credential_filename}') as kaggle_file:
    kaggle_cred = json.load(kaggle_file)

kaggle_username = kaggle_cred['username']
kaggle_key = kaggle_cred['key']

In [3]:
import sys
import os
modules = dir()

modules

['In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'gdown',
 'get_ipython',
 'modules',
 'np',
 'os',
 'pd',
 'plt',
 'quit',
 'sys',
 'time',
 'zipfile']

In [ ]:
# [TODO] Query & download metadata


In [4]:
############################################################ 
#################### Getting the metadata ####################
######################################## ####################



Starting Download
Access denied with the following error:



 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=0B7EVK8r0v71pWnFiNlNGTVloLUk 



In [ ]:
# [TODO] Query & download images

In [ ]:
############################################################ 
#################### Getting the actual images ####################
######################################## ####################

root_path = './'
url = 'https://drive.google.com/uc?id=1j5fCPgh0gnY6v7ChkWlgnnHH6unxuAbb'
output = 'img.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile("img.zip","r") as zip_ref:
    zip_ref.extractall(root_path)

In [ ]:
# [TODO] Once we get images & metadata, combine

In [ ]:
############################################################ 
#################### Creating the master dataset ####################
######################################## ####################


category_list = []
image_path_list = []
data_type_list = []
# category names
with open('list_category_cloth.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1:
            category_list.append(line.split(' ')[0])

# category map
with open('list_category_img.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1:
            image_path_list.append([word.strip() for word in line.split(' ') if len(word) > 0])


# train, valid, test
with open('list_eval_partition.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1:
            data_type_list.append([word.strip() for word in line.split(' ') if len(word) > 0])

In [ ]:
data_df = pd.DataFrame(image_path_list, columns=['image_path', 'category_number'])
data_df['category_number'] = data_df['category_number'].astype(int)
data_df = data_df.merge(pd.DataFrame(data_type_list, columns=['image_path', 'dataset_type']), on='image_path')
data_df['category'] = data_df['category_number'].apply(lambda x: category_list[int(x) - 1])
data_df = data_df.drop('category_number', axis=1)
data_df.head()

In [ ]:
# [TODO] Do initial EDA 

In [ ]:
############################################################ 
#################### EDA ####################
######################################## ####################

len(data_df)

In [ ]:
############################################################ 
#################### EDA ####################
######################################## ####################

data_df[['image_path','dataset_type']].groupby('dataset_type').count()

In [ ]:
############################################################ 
#################### EDA ####################
######################################## ####################


len(data_df.category.unique())

In [ ]:
############################################################ 
#################### EDA ####################
######################################## ####################



data_df[['image_path','category']].groupby('category').count()